In [11]:
import nltk
nltk.download('punkt')
nltk.download('tagsets')
nltk.help.upenn_tagset()
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
import pandas as pd

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\edwardsrk\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package tagsets to
[nltk_data]     C:\Users\edwardsrk\AppData\Roaming\nltk_data...
[nltk_data]   Package tagsets is already up-to-date!


$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
IN: preposition or

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer

In [3]:
count = CountVectorizer()
"""CountVectorizer(
    *,
    input='content',
    encoding='utf-8',
    decode_error='strict',
    strip_accents=None,
    lowercase=True,
    preprocessor=None,
    tokenizer=None,
    stop_words=None,
    token_pattern='(?u)\\b\\w\\w+\\b',
    ngram_range=(1, 1),
    analyzer='word',
    max_df=1.0,
    min_df=1,
    max_features=None,
    vocabulary=None,
    binary=False,
    dtype=<class 'numpy.int64'>,"""
#Convert a collection of text documents to a matrix of token counts
#This implementation produces a sparse representation of the counts using
#scipy.sparse.csr_matrix.
#If you do not provide an a-priori dictionary and you do not use an analyzer
#that does some kind of feature selection then the number of features will
#be equal to the vocabulary size found by analyzing the data.

tfidf = TfidfVectorizer()
"""TfidfVectorizer(
    *,
    input='content',
    encoding='utf-8',
    decode_error='strict',
    strip_accents=None,
    lowercase=True,
    preprocessor=None,
    tokenizer=None,
    analyzer='word',
    stop_words=None,
    token_pattern='(?u)\\b\\w\\w+\\b',
    ngram_range=(1, 1),
    max_df=1.0,
    min_df=1,
    max_features=None,
    vocabulary=None,
    binary=False,
    dtype=<class 'numpy.float64'>,
    norm='l2',
    use_idf=True,
    smooth_idf=True,
    sublinear_tf=False,"""
#Convert a collection of raw documents to a matrix of TF-IDF features.

hashing = HashingVectorizer()
"""HashingVectorizer(
    *,
    input='content',
    encoding='utf-8',
    decode_error='strict',
    strip_accents=None,
    lowercase=True,
    preprocessor=None,
    tokenizer=None,
    stop_words=None,
    token_pattern='(?u)\\b\\w\\w+\\b',
    ngram_range=(1, 1),
    analyzer='word',
    n_features=1048576,
    binary=False,
    norm='l2',
    alternate_sign=True,
    dtype=<class 'numpy.float64'>,
)"""

"""Convert a collection of text documents to a matrix of token occurrences

It turns a collection of text documents into a scipy.sparse matrix holding
token occurrence counts (or binary occurrence information), possibly
normalized as token frequencies if norm='l1' or projected on the euclidean
unit sphere if norm='l2'.

This text vectorizer implementation uses the hashing trick to find the
token string name to feature integer index mapping.

This strategy has several advantages:

- it is very low memory scalable to large datasets as there is no need to
  store a vocabulary dictionary in memory

- it is fast to pickle and un-pickle as it holds no state besides the
  constructor parameters

- it can be used in a streaming (partial fit) or parallel pipeline as there
  is no state computed during fit.

There are also a couple of cons (vs using a CountVectorizer with an
in-memory vocabulary):

- there is no way to compute the inverse transform (from feature indices to
  string feature names) which can be a problem when trying to introspect
  which features are most important to a model.

- there can be collisions: distinct tokens can be mapped to the same
  feature index. However in practice this is rarely an issue if n_features
  is large enough (e.g. 2 ** 18 for text classification problems).

- no IDF weighting as this would render the transformer stateful.

The hash function employed is the signed 32-bit version of Murmurhash3."""

"Convert a collection of text documents to a matrix of token occurrences\n\nIt turns a collection of text documents into a scipy.sparse matrix holding\ntoken occurrence counts (or binary occurrence information), possibly\nnormalized as token frequencies if norm='l1' or projected on the euclidean\nunit sphere if norm='l2'.\n\nThis text vectorizer implementation uses the hashing trick to find the\ntoken string name to feature integer index mapping.\n\nThis strategy has several advantages:\n\n- it is very low memory scalable to large datasets as there is no need to\n  store a vocabulary dictionary in memory\n\n- it is fast to pickle and un-pickle as it holds no state besides the\n  constructor parameters\n\n- it can be used in a streaming (partial fit) or parallel pipeline as there\n  is no state computed during fit.\n\nThere are also a couple of cons (vs using a CountVectorizer with an\nin-memory vocabulary):\n\n- there is no way to compute the inverse transform (from feature indices to\

In [4]:
def get_wordnet_pos(treebank_tag):
    '''
    Translate nltk POS to wordnet tags
    '''
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN


def data_cleaner(doc):
    """A function to strip punctuation, strip stopwords, casefold, lemmatize,
    And part pf speech tag words for clean data for modeling"""
    
    sw = stopwords.words('english')
    regex_token = RegexpTokenizer(r"([a-zA-Z]+(?:’[a-z]+)?)")
    doc = regex_token.tokenize(doc)
    doc = [word.lower() for word in doc]
    doc = [word for word in doc if word not in sw]
    #print(doc)
    doc = pos_tag(doc)
    doc = [(word[0], get_wordnet_pos(word[1])) for word in doc]
    #print(doc)
    lemmatizer = WordNetLemmatizer() 
    doc = [lemmatizer.lemmatize(word[0], word[1]) for word in doc]
    #print(' '.join(doc))
    return ' '.join(doc)

In [13]:
text = {'sent':["I love to eat ice cream", "Ice cream is my favorite thing to be eating", "everyone is loving eating the ice cream"]}

In [14]:
df = pd.DataFrame(text)
df

,sent
0,I love to eat ice cream
1,Ice cream is my favorite thing to be eating
2,everyone is loving eating the ice cream


In [15]:
c_list = df.sent.tolist()

clean_corpus = []
for doc in c_list:
    clean_corpus.append(data_cleaner(doc))

In [18]:
clean_corpus
df['clean'] = clean_corpus

In [19]:
df

,sent,clean
0,I love to eat ice cream,love eat ice cream
1,Ice cream is my favorite thing to be eating,ice cream favorite thing eating
2,everyone is loving eating the ice cream,everyone love eat ice cream


In [25]:
tt_count = count.fit_transform(df.clean)
tt_count  = pd.DataFrame.sparse.from_spmatrix(tt_count)
tt_count.columns = sorted(count.vocabulary_)
tt_count

,cream,eat,eating,everyone,favorite,ice,love,thing
0,1,1,0,0,0,1,1,0
1,1,0,1,0,1,1,0,1
2,1,1,0,1,0,1,1,0


In [26]:
tt_tfidf = tfidf.fit_transform(df.clean)
tt_tfidf  = pd.DataFrame.sparse.from_spmatrix(tt_tfidf)
tt_tfidf.columns = sorted(count.vocabulary_)
tt_tfidf

,cream,eat,eating,everyone,favorite,ice,love,thing
0,0.433708,0.558478,0.00000,0.000000,0.00000,0.433708,0.558478,0.00000
1,0.307144,0.000000,0.52004,0.000000,0.52004,0.307144,0.000000,0.52004
2,0.349578,0.450145,0.00000,0.591887,0.00000,0.349578,0.450145,0.00000


In [34]:
tt_hashing = hashing.fit_transform(df.clean)
#tt_hashing
tt_hashing  = pd.DataFrame.sparse.from_spmatrix(tt_hashing)
tt_hashing
#tt_hashing.columns = sorted(count.vocabulary_)
#tt_hashing

,0,1,2,3,4,5,6,7,8,9,...,1048566,1048567,1048568,1048569,1048570,1048571,1048572,1048573,1048574,1048575
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
